# Kaggle 3: Sentence-level Substring Matching vs Regex

In [1]:

import src.models.kaggle_model3 as km3
import src.models.kaggle_model3_regex_inference as km3r
import src.evaluate.model as em
from src.data.kaggle_repository import KaggleRepository

In [2]:
config = {
    "model_path": "../models/kaggle_model3/baseline/params.txt",
}

repo = KaggleRepository()

In [3]:
evaluation = em.evaluate_model(
    repo,
    km3.KaggleModel3(),
    config,
)
evaluation


        Model Evaluation:

        - Run time: 42.66263294219971 seconds, avg: 0.005291161223142714 seconds per sample
        - True Postive Count: 2446, avg: 0.3033610318739923 per sample
        - Precision: 0.9012527634487841
        - Recall: 0.06857110818311794
        

In [5]:
# import pandas as pd

# merged_df = pd.DataFrame({
#     "id": stats.loc[:, ["id"]].values.flatten(),
# })

# merged_df["label"] = merged_df["id"].apply(
#     lambda _id: stats.loc[stats["id"] == _id, "label"].values[0]
# )


In [16]:
# def merge_single_stat(this:str, that:str, lbl:str) -> str:
#     if this == "TP" or that == "TP":
#         return "TP"
#     elif this != that:
#         raise ValueError("Inconsistent stats for label: {}. got {} - {}".format(lbl, this, that))
#     else:
#         return this


# def merge_stats(row:pd.DataFrame):
#     _id = row["id"]
#     self_labels = row["statistics_self"]["labels"]
#     other_labels = row["statistics_other"]["labels"]
#     self_stats = row["statistics_self"]["stats"]
#     other_stats = row["statistics_other"]["stats"]

#     merged_lbls = []
#     merged_stats = []
#     for self_lbl in self_labels:
#         self_stat = self_stats[self_labels.index(self_lbl)]
#         merged_lbls.append(self_lbl)
        
#         if self_lbl in other_labels:
#             merged_stats.append(merge_single_stat(
#                 self_stat, 
#                 other_stats[other_labels.index(self_lbl)], 
#                 self_lbl,
#             ))
#         else:
#             merged_stats.append(self_stat)

#     for other_lbl in set(other_labels) - set(self_labels):
#         other_stat = other_stats[other_labels.index(other_lbl)]
#         merged_lbls.append(other_lbl)
#         merged_stats.append(other_stat)

#     return {"id":_id, "statistics": dict(labels=merged_lbls, stats=merged_stats)}

# stats.loc[:, ["id", "statistics"]].merge(
#     stats.loc[:, ["id", "statistics"]],
#     on="id",
#     suffixes=("_self", "_other"),
# ).apply(merge_stats, axis=1, result_type="expand")

,id,statistics
0,7bfd8bb51,"{'labels': ['dbgap'], 'stats': ['FN']}"
1,7d3e31302,"{'labels': ['sass', 'sass data', 'school and s..."
2,3644f959a,"{'labels': ['arms', 'arms data', 'agricultural..."
3,ed3527cf3,{'labels': ['database of genotypes and phenoty...
4,236061129,"{'labels': ['dbgap'], 'stats': ['FN']}"
...,...,...
8058,3cb8b0e09,{'labels': ['database of genotypes and phenoty...
8059,324fe1310,"{'labels': ['dbgap'], 'stats': ['FN']}"
8060,39e66a274,"{'labels': ['1000 genomes project', '1000 geno..."
8061,c411b1b6c,"{'labels': ['dbgap', 'american cancer society ..."


In [4]:
evaluation_regex = em.evaluate_model(
    repo,
    km3r.Kaggle3RegexInference(config),
    config,
)
evaluation_regex


        Model Evaluation:

        - Run time: 723.7678065299988 seconds, avg: 0.08976408365744745 seconds per sample
        - True Postive Count: 6977, avg: 0.8653106784075406 per sample
        - Precision: 0.9998566924620235
        - Recall: 0.19591710659328315
        

In [5]:
evaluation | evaluation_regex


        Model Evaluation:

        - Run time: 766.4304394721985 seconds, avg: 0.09505524488059017 seconds per sample
        - True Postive Count: 7041, avg: 0.8732481706560833 per sample
        - Precision: 0.9632010943912449
        - Recall: 0.19738723332679206
        